In [1]:
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('cpu_times.csv')
df['File'] = df['File'].str.replace(r'\.py_time.*', '', regex=True)
df.to_csv('cpu_times_cleaned.csv', index=False)

df = pd.read_csv('system_calls.csv')
df['File'] = df['File'].str.replace(r'\.py_strace.*', '', regex=True)
df.to_csv('system_calls_cleaned.csv', index=False)

In [3]:
df = pd.read_csv('system_calls_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Columns: 126 entries, File to checkPrime
dtypes: int64(125), object(1)
memory usage: 174.4+ KB


In [4]:
column_info = df.dtypes
print("Column name      Data Type")
for column, dtype in column_info.items():
    print(f"{column}                  {dtype}")

Column name      Data Type
File                  object
getdents64                  int64
rfind                  int64
str                  int64
pread64                  int64
exit_group                  int64
lit                  int64
rob                  int64
sum                  int64
DivisibleByThree                  int64
poll                  int64
svRead                  int64
insertionSort                  int64
rt_sigaction                  int64
log                  int64
mremap                  int64
mprotect                  int64
factorial                  int64
map                  int64
fib                  int64
_occr_element                  int64
int                  int64
pop                  int64
sendto                  int64
uname                  int64
getpid                  int64
range                  int64
htons                  int64
tprint                  int64
lay                  int64
mmap                  int64
346z                  int64
getsockopt

In [5]:
df = pd.read_csv('cpu_times_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   File                173 non-null    object 
 1   CPU Time (seconds)  173 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


In [6]:
df = pd.read_csv('system_calls_cleaned.csv')

# List of columns to remove
columns_to_remove = [
    'rfind', 'str', 'lit', 'rob', 'sum', 'DivisibleByThree', 'svRead', 'insertionSort',
    'rt_sigaction', 'log', 'mremap', 'mprotect', 'factorial', 'map', 'fib', '_occr_element',
    'int', 'pop', 'tprint', 'lay', '346z', 'it', 'rseq', 'rint', 'nreverse_a_string',
    'csvRead', 'main', 'st', 'lseek', 'arch_prctl', 'read', 'sysinfo', 'findEquilibrium',
    'auth', 'firstRepeatedChar', 'sendmmsg', 'if_nametoindex', '217', 'len', 'join',
    'nprint', 'nscan_network', '334', 'list', 'eckGreater', 'munmap', 'getsockname',
    'lper', 'login', 'argumentParser', '323', 'makedev', 'divide', 'myFunction', 'peek',
    'd', 'newfstatat', 't', 'inet_pton', 'recvfrom', 'swapPositions', 'RemoveDuplicate',
    'arySearch', 'add', 'inet_addr', 'shellSort', 'set_tid_address', 'set_robust_list',
    'readlink', 'mirror', 'input', 'htonl', 'ndmail', 'rd', 'prlimit64', 'count_alphabets',
    'tMissingPositive', 'epoll_create1', '202', 'getpeername', 'checkPrime'
]

# Remove the specified columns
df = df.drop(columns=columns_to_remove, errors='ignore')

# Save the modified DataFrame back to a new CSV file
df.to_csv('system_calls_cleaned_updated.csv', index=False)


In [7]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor

In [20]:
# Load the dataset
data = pd.read_csv("SCE3.csv")

In [10]:
# Handle missing values in the target variable 'CPU Time (seconds)'
y = data["CPU Time (seconds)"]
y.fillna(y.mean(), inplace=True)  # Impute missing values with the mean

In [11]:
# Drop the target variable from the feature matrix
X = data.drop("CPU Time (seconds)", axis=1)

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Initialize base models
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, random_state=42)
lgbm_model = LGBMRegressor(n_estimators=1000, max_depth=6, learning_rate=0.1, random_state=42)

In [14]:
# Initialize the stacking ensemble model
stacked_model = StackingRegressor(
    estimators=[('catboost', catboost_model), ('lgbm', lgbm_model)],
    final_estimator=LGBMRegressor(n_estimators=100, max_depth=13,num_leaves=38, learning_rate=0.1, random_state=42),
)

In [15]:
# Train the stacking model on the training data
stacked_model.fit(X_train, y_train)

# Make predictions using the stacking model
stacked_predictions = stacked_model.predict(X_test)

C:\Users\Gayatri\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Gayatri\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


0:	learn: 3.0314280	total: 143ms	remaining: 2m 23s
1:	learn: 3.0169752	total: 144ms	remaining: 1m 11s
2:	learn: 3.0040995	total: 145ms	remaining: 48.1s
3:	learn: 3.0009169	total: 145ms	remaining: 36.1s
4:	learn: 2.9983467	total: 145ms	remaining: 28.9s
5:	learn: 2.9822489	total: 146ms	remaining: 24.2s
6:	learn: 2.9708326	total: 146ms	remaining: 20.8s
7:	learn: 2.9688050	total: 147ms	remaining: 18.2s
8:	learn: 2.9675379	total: 147ms	remaining: 16.2s
9:	learn: 2.9661572	total: 148ms	remaining: 14.6s
10:	learn: 2.9653191	total: 148ms	remaining: 13.3s
11:	learn: 2.9646048	total: 149ms	remaining: 12.2s
12:	learn: 2.9633570	total: 149ms	remaining: 11.3s
13:	learn: 2.9613475	total: 150ms	remaining: 10.5s
14:	learn: 2.9609669	total: 150ms	remaining: 9.85s
15:	learn: 2.9489366	total: 151ms	remaining: 9.26s
16:	learn: 2.9384911	total: 151ms	remaining: 8.74s
17:	learn: 2.9382248	total: 151ms	remaining: 8.27s
18:	learn: 2.9376072	total: 152ms	remaining: 7.84s
19:	learn: 2.9368905	total: 152ms	remai

312:	learn: 2.7963806	total: 317ms	remaining: 696ms
313:	learn: 2.7963781	total: 318ms	remaining: 695ms
314:	learn: 2.7963603	total: 319ms	remaining: 693ms
315:	learn: 2.7963213	total: 319ms	remaining: 691ms
316:	learn: 2.7963191	total: 320ms	remaining: 689ms
317:	learn: 2.7963111	total: 320ms	remaining: 687ms
318:	learn: 2.7962500	total: 321ms	remaining: 685ms
319:	learn: 2.7962448	total: 321ms	remaining: 683ms
320:	learn: 2.7962374	total: 322ms	remaining: 681ms
321:	learn: 2.7962312	total: 323ms	remaining: 679ms
322:	learn: 2.7962271	total: 323ms	remaining: 678ms
323:	learn: 2.7962215	total: 324ms	remaining: 676ms
324:	learn: 2.7962052	total: 324ms	remaining: 674ms
325:	learn: 2.7961675	total: 325ms	remaining: 672ms
326:	learn: 2.7961541	total: 326ms	remaining: 671ms
327:	learn: 2.7961230	total: 327ms	remaining: 669ms
328:	learn: 2.7961128	total: 327ms	remaining: 667ms
329:	learn: 2.7961095	total: 328ms	remaining: 665ms
330:	learn: 2.7961081	total: 328ms	remaining: 663ms
331:	learn: 

616:	learn: 2.7920079	total: 491ms	remaining: 305ms
617:	learn: 2.7920072	total: 492ms	remaining: 304ms
618:	learn: 2.7920043	total: 492ms	remaining: 303ms
619:	learn: 2.7919722	total: 493ms	remaining: 302ms
620:	learn: 2.7919604	total: 493ms	remaining: 301ms
621:	learn: 2.7919589	total: 494ms	remaining: 300ms
622:	learn: 2.7919372	total: 494ms	remaining: 299ms
623:	learn: 2.7919370	total: 495ms	remaining: 298ms
624:	learn: 2.7919292	total: 495ms	remaining: 297ms
625:	learn: 2.7919277	total: 496ms	remaining: 296ms
626:	learn: 2.7919258	total: 497ms	remaining: 295ms
627:	learn: 2.7919243	total: 497ms	remaining: 295ms
628:	learn: 2.7919235	total: 498ms	remaining: 294ms
629:	learn: 2.7919218	total: 498ms	remaining: 293ms
630:	learn: 2.7919184	total: 499ms	remaining: 292ms
631:	learn: 2.7919097	total: 499ms	remaining: 291ms
632:	learn: 2.7919069	total: 500ms	remaining: 290ms
633:	learn: 2.7919060	total: 500ms	remaining: 289ms
634:	learn: 2.7918969	total: 501ms	remaining: 288ms
635:	learn: 

940:	learn: 2.7910111	total: 668ms	remaining: 41.9ms
941:	learn: 2.7910111	total: 669ms	remaining: 41.2ms
942:	learn: 2.7910080	total: 670ms	remaining: 40.5ms
943:	learn: 2.7910079	total: 670ms	remaining: 39.8ms
944:	learn: 2.7910078	total: 671ms	remaining: 39ms
945:	learn: 2.7910078	total: 671ms	remaining: 38.3ms
946:	learn: 2.7910076	total: 672ms	remaining: 37.6ms
947:	learn: 2.7910068	total: 673ms	remaining: 36.9ms
948:	learn: 2.7910024	total: 673ms	remaining: 36.2ms
949:	learn: 2.7909997	total: 674ms	remaining: 35.5ms
950:	learn: 2.7909996	total: 674ms	remaining: 34.7ms
951:	learn: 2.7909991	total: 675ms	remaining: 34ms
952:	learn: 2.7909990	total: 675ms	remaining: 33.3ms
953:	learn: 2.7909905	total: 676ms	remaining: 32.6ms
954:	learn: 2.7909894	total: 676ms	remaining: 31.9ms
955:	learn: 2.7909830	total: 677ms	remaining: 31.2ms
956:	learn: 2.7909823	total: 677ms	remaining: 30.4ms
957:	learn: 2.7909802	total: 678ms	remaining: 29.7ms
958:	learn: 2.7909793	total: 679ms	remaining: 29ms

...

581:	learn: 2.3872053	total: 607ms	remaining: 436ms
582:	learn: 2.3872040	total: 608ms	remaining: 435ms
583:	learn: 2.3871676	total: 609ms	remaining: 434ms
584:	learn: 2.3871671	total: 610ms	remaining: 433ms
585:	learn: 2.3871632	total: 612ms	remaining: 432ms
586:	learn: 2.3871621	total: 613ms	remaining: 431ms
587:	learn: 2.3871618	total: 613ms	remaining: 430ms
588:	learn: 2.3871612	total: 615ms	remaining: 429ms
589:	learn: 2.3871609	total: 615ms	remaining: 428ms
590:	learn: 2.3871593	total: 616ms	remaining: 427ms
591:	learn: 2.3871251	total: 618ms	remaining: 426ms
592:	learn: 2.3871202	total: 619ms	remaining: 425ms
593:	learn: 2.3871187	total: 620ms	remaining: 424ms
594:	learn: 2.3871095	total: 621ms	remaining: 423ms
595:	learn: 2.3871069	total: 622ms	remaining: 422ms
596:	learn: 2.3870737	total: 623ms	remaining: 421ms
597:	learn: 2.3870673	total: 624ms	remaining: 420ms
598:	learn: 2.3870647	total: 625ms	remaining: 419ms
599:	learn: 2.3870581	total: 626ms	remaining: 417ms
600:	learn: 

751:	learn: 2.3865499	total: 776ms	remaining: 256ms
752:	learn: 2.3865480	total: 779ms	remaining: 256ms
753:	learn: 2.3865459	total: 780ms	remaining: 255ms
754:	learn: 2.3865442	total: 781ms	remaining: 254ms
755:	learn: 2.3865434	total: 782ms	remaining: 253ms
756:	learn: 2.3865428	total: 783ms	remaining: 251ms
757:	learn: 2.3865427	total: 784ms	remaining: 250ms
758:	learn: 2.3865415	total: 785ms	remaining: 249ms
759:	learn: 2.3865413	total: 786ms	remaining: 248ms
760:	learn: 2.3865413	total: 787ms	remaining: 247ms
761:	learn: 2.3865400	total: 788ms	remaining: 246ms
762:	learn: 2.3865389	total: 789ms	remaining: 245ms
763:	learn: 2.3865377	total: 790ms	remaining: 244ms
764:	learn: 2.3865375	total: 791ms	remaining: 243ms
765:	learn: 2.3865369	total: 791ms	remaining: 242ms
766:	learn: 2.3865365	total: 792ms	remaining: 241ms
767:	learn: 2.3865364	total: 792ms	remaining: 239ms
768:	learn: 2.3865358	total: 801ms	remaining: 241ms
769:	learn: 2.3865346	total: 802ms	remaining: 240ms
770:	learn: 

955:	learn: 2.3863605	total: 953ms	remaining: 43.9ms
956:	learn: 2.3863602	total: 954ms	remaining: 42.9ms
957:	learn: 2.3863600	total: 954ms	remaining: 41.8ms
958:	learn: 2.3863599	total: 955ms	remaining: 40.8ms
959:	learn: 2.3863599	total: 956ms	remaining: 39.8ms
960:	learn: 2.3863584	total: 957ms	remaining: 38.8ms
961:	learn: 2.3863581	total: 958ms	remaining: 37.8ms
962:	learn: 2.3863579	total: 958ms	remaining: 36.8ms
963:	learn: 2.3863576	total: 959ms	remaining: 35.8ms
964:	learn: 2.3863575	total: 960ms	remaining: 34.8ms
965:	learn: 2.3863574	total: 960ms	remaining: 33.8ms
966:	learn: 2.3863574	total: 961ms	remaining: 32.8ms
967:	learn: 2.3863574	total: 961ms	remaining: 31.8ms
968:	learn: 2.3863571	total: 962ms	remaining: 30.8ms
969:	learn: 2.3863571	total: 963ms	remaining: 29.8ms
970:	learn: 2.3863570	total: 964ms	remaining: 28.8ms
971:	learn: 2.3863567	total: 964ms	remaining: 27.8ms
972:	learn: 2.3863566	total: 965ms	remaining: 26.8ms
973:	learn: 2.3863565	total: 965ms	remaining: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83
[LightGBM] [Info] Number of data points in the train set: 177, number of used features: 15
[LightGBM] [Info] Start training from score 0.695003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

In [16]:
# Evaluate the stacking model
mse = mean_squared_error(y_test, stacked_predictions)
rmse = mse**0.5
r2 = r2_score(y_test, stacked_predictions)

print("Mean Squared Error (Stacking Hybrid):", mse)
print("Root Mean Squared Error (Stacking Hybrid):", rmse)
print("R-squared (Stacking Hybrid):", r2)

Mean Squared Error (Stacking Hybrid): 3.0198682608020024
Root Mean Squared Error (Stacking Hybrid): 1.737776815589966
R-squared (Stacking Hybrid): -0.3235469520430192


C:\Users\Gayatri\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Gayatri\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Gayatri\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Gayatri\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(d

In [17]:
# Create a DataFrame with the new input data
datas = {
    'getgid': [1],
    'getrandom': [2],
    'access': [1],
    'getdents64': [18],
    'fcntl': [1],
    'write': [0],
    'getcwd': [2],
    'geteuid': [1],
    'execve': [1],
    'dup': [3],
    'mmap': [27],
    'prctl': [0],
    'gettid': [0],
    'print': [0],
    'ioctl': [0],
    'openat': [24],
    'getuid': [37],
    'getegid': [1],
    'brk': [1],
    'pread64': [10],
    'isSorted': [4],
    'futex': [0],
    'close': [1],
    'exit_group': [37]
}

df = pd.DataFrame(datas)
stacked_predictions = stacked_model.predict(df)

# Print the predicted CPU Time (seconds) for the new data
print("Predicted CPU Time (seconds) for new data:", stacked_predictions[0])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Predicted CPU Time (seconds) for new data: 0.059790626632380106


In [21]:
correlation = data.corr()[r'CPU Time (seconds)']

In [22]:
correlation

getgid                     NaN
getrandom             0.103635
access                0.123586
getdents64            0.122781
fcntl                 0.117154
write                -0.001795
getcwd                0.123586
geteuid                    NaN
execve                     NaN
dup                        NaN
mmap                  0.123278
prctl                 0.123586
gettid                0.123586
print                 0.048882
ioctl                 0.104015
openat                0.122898
getuid                     NaN
getegid                    NaN
brk                   0.126864
pread64                    NaN
isSorted             -0.013930
futex                 0.122572
close                 0.122889
exit_group            0.013251
CPU Time (seconds)    1.000000
Name: CPU Time (seconds), dtype: float64

In [23]:
with open('CPU_exe_model.pkl', 'wb') as file:
    pickle.dump(stacked_model, file)